In [1]:
import xarray as xr 

# ds = xr.open_dataset('/teamspace/studios/this_studio/ml-drought-forecasting/ml-modeling-pipeline/data/01_raw/f0e93607df96295f1c9e14ccfe96032f.nc')
ds = xr.open_dataset('/Users/adamprzychodni/Documents/Repos/ml-drought-forecasting/ml-modeling-pipeline/data/01_raw/ERA5_monthly_averaged_data_on_single_levels.nc')

In [2]:
ds

<xarray.Dataset> Size: 44GB
Dimensions:    (date: 405, latitude: 721, longitude: 1440)
Coordinates:
    number     int64 8B ...
  * date       (date) int64 3kB 19910101 19910201 19910301 ... 20240801 20240901
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    expver     (date) <U4 6kB ...
Data variables: (12/26)
    u10        (date, latitude, longitude) float32 2GB ...
    v10        (date, latitude, longitude) float32 2GB ...
    t2m        (date, latitude, longitude) float32 2GB ...
    sst        (date, latitude, longitude) float32 2GB ...
    sp         (date, latitude, longitude) float32 2GB ...
    tp         (date, latitude, longitude) float32 2GB ...
    ...         ...
    lai_lv     (date, latitude, longitude) float32 2GB ...
    cvl        (date, latitude, longitude) float32 2GB ...
    tvh        (date, latitude, longitude) float32 2GB ...
    tvl        (date, latitude, longitude) float32 2GB ...
    z          (date, latitude, longitude) float32 2GB ...
    lsm        (date, latitude, longitude) float32 2GB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-25T11:10 GRIB to CDM+CF via cfgrib-0.9.1...

In [3]:
import pandas as pd

# 1. Convert 'date' coordinate to datetime
ds['date'] = pd.to_datetime(ds['date'].astype(str), format='%Y%m%d')

In [4]:
import numpy as np
import xarray as xr

# Step 1: Define target grid with 1° resolution
target_lat = np.arange(-90, 90.1, 1)   # From -90 to 90 degrees inclusive
target_lon = np.arange(0, 360, 1)      # From 0 to 359.5 degrees inclusive

# Step 2: Create target grid Dataset (optional, for reference)
target_grid = xr.Dataset(
    {
        "latitude": (["latitude"], target_lat),
        "longitude": (["longitude"], target_lon),
    }
)

# Step 3: Ensure latitude is ascending
if ds.latitude[0] > ds.latitude[-1]:
    ds = ds.sortby("latitude")

# Step 4: Perform interpolation
ds = ds.interp(latitude=target_lat, longitude=target_lon, method="linear")


In [5]:
ds

<xarray.Dataset> Size: 5GB
Dimensions:    (date: 405, latitude: 181, longitude: 360)
Coordinates:
    number     int64 8B 0
  * date       (date) datetime64[ns] 3kB 1991-01-01 1991-02-01 ... 2024-09-01
    expver     (date) <U4 6kB '0001' '0001' '0001' ... '0001' '0005' '0005'
  * latitude   (latitude) float64 1kB -90.0 -89.0 -88.0 -87.0 ... 88.0 89.0 90.0
  * longitude  (longitude) int64 3kB 0 1 2 3 4 5 6 ... 354 355 356 357 358 359
Data variables: (12/26)
    u10        (date, latitude, longitude) float64 211MB 0.3863 0.3863 ... -1.83
    v10        (date, latitude, longitude) float64 211MB 0.07195 ... -0.2351
    t2m        (date, latitude, longitude) float64 211MB 244.2 244.2 ... 267.0
    sst        (date, latitude, longitude) float64 211MB nan nan ... 271.5 271.5
    sp         (date, latitude, longitude) float64 211MB 6.901e+04 ... 1.015e+05
    tp         (date, latitude, longitude) float64 211MB 6.294e-05 ... 0.000576
    ...         ...
    lai_lv     (date, latitude, longitude) float64 211MB 0.0 0.0 0.0 ... 0.0 0.0
    cvl        (date, latitude, longitude) float64 211MB 0.0 0.0 0.0 ... 0.0 0.0
    tvh        (date, latitude, longitude) float64 211MB 0.0 0.0 0.0 ... 0.0 0.0
    tvl        (date, latitude, longitude) float64 211MB 0.0 0.0 0.0 ... 0.0 0.0
    z          (date, latitude, longitude) float64 211MB 2.736e+04 ... -1.24
    lsm        (date, latitude, longitude) float64 211MB 1.0 1.0 1.0 ... 0.0 0.0
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-25T11:10 GRIB to CDM+CF via cfgrib-0.9.1...

In [7]:
# ds.to_netcdf('/Users/adamprzychodni/Documents/Repos/ml-drought-forecasting/ml-modeling-pipeline/data/02_intermediate/merged_data.nc')
ds.to_netcdf('/Users/adamprzychodni/Documents/Repos/ml-drought-forecasting/ml-modeling-pipeline/data/02_intermediate/preprocessed_data.nc')